# Label 2-photon retinal images 

This notebook is used to label 2 photon retinal images

## Define helper to convert bounding box format

Eventually this should be in a helper

In [37]:
import numpy as np

def x1y1x2y2_to_tltrblbr(x1, y1, x2, y2, n =-1):
    """
    Convert a bounding box defined by its top-left and bottom-right coordinates to the 
    coordinates of its corners in pixel space (top-left, top-right, bottom-right, bottom-left).

    Parameters:
    - x1 (int): x-coordinate of the top-left corner of the bounding box.
    - y1 (int): y-coordinate of the top-left corner of the bounding box.
    - x2 (int): x-coordinate of the bottom-right corner of the bounding box.
    - y2 (int): y-coordinate of the bottom-right corner of the bounding box.

    Returns:
    - np.ndarray: Array of shape (4, 2) containing the coordinates of the bounding box corners 
      in the format [[top_left_y, top_left_x], [top_right_y, top_right_x], 
      [bottom_right_y, bottom_right_x], [bottom_left_y, bottom_left_x]].
    """
    if n == -1:
        return np.array([[y1, x1], [y1, x2], [y2, x2], [y2, x1]])
    else: 
      return np.array([[n, y1, x1], [n, y1, x2], [n, y2, x2], [n, y2, x1]]) 

## Open Image and (possibly) pre-existing label

In [38]:
from skimage.io import imread, imsave
import napari
import os
import numpy as np
from pathlib import Path
from tnia.deeplearning.dl_helper import make_label_directory
from glob import glob
import json

# define tnia images path
#tnia_images_path = Path(r'D:/images/tnia-python-images')
tnia_images_path = Path(r'/home/bnorthan/images/tnia-python-images')

# define parent path of project and label and image path
parent_path=Path(tnia_images_path / r'imagesc/2024_08_08_2photon_vessel')
label_path = Path(parent_path / r'labels')
image_path = Path(parent_path )

name = 'image1'
ext = '.jpg'

image_label_paths, mask_label_paths = make_label_directory(1, 1, label_path)

# open image that we will label
im = imread(os.path.join(image_path / (name+ext)))

c = 0 

# get list of tif files in image_label_path (these correspond to previously existing labels)
label_names = list(Path(mask_label_paths[c]).glob('*.tif'))
json_names = list(Path(image_label_paths[c]).glob('*.json'))
                
# get all label names for this image
label_names_ = [x for x in label_names if name in x.name]
json_names_ = [x for x in json_names if name in x.name]

# sort the label names and json names to make sure they correspond
label_names_ = sorted(label_names_)
json_names_ = sorted(json_names_)

try:
    label = imread(os.path.join(mask_label_path / (name+".tif")))
except:
    print('labels not created yet', (os.path.join(label_path / (name+".tif"))))
    label = np.zeros([im.shape[0], im.shape[1]], dtype=np.uint8)

labels_temp = []
boxes = []
n = 0

for label_name_, json_name_ in zip(label_names_, json_names_):

    print('label name is ', label_name_)
    print('json name is ', json_name_)

    with open(json_name_, 'r') as f:
        json_ = json.load(f)
        print(json_)
                                                        
        x1= json_['bbox'][0]
        y1= json_['bbox'][1]
        x2= json_['bbox'][2]
        y2= json_['bbox'][3]

        bbox = x1y1x2y2_to_tltrblbr(x1, y1, x2, y2, -1)
        boxes.append(bbox)

        label_crop = imread(label_name_)
        label[y1:y2, x1:x2] = label_crop
        #rois.append([[x1, y1], [x2, y2]])

        labels_temp.append(label)



labels not created yet /home/bnorthan/images/tnia-python-images/imagesc/2024_08_08_2photon_vessel/labels/image1.tif
label name is  /home/bnorthan/images/tnia-python-images/imagesc/2024_08_08_2photon_vessel/labels/ground truth0/image1_0.tif
json name is  /home/bnorthan/images/tnia-python-images/imagesc/2024_08_08_2photon_vessel/labels/input0/image1_0.json
{'base_name': 'image1_0', 'bbox': [192, 487, 738, 748]}


## Edit an ROI of the labels in Napari

Now edit the labels in Napari.  We can start from a blank image or alternatively a pre-existing set of labels (either generated by another method, or previous manual labels) that we want to improve.

Use the box ROI to indicate which region of the image contains labels.

In [39]:
viewer = napari.Viewer()
viewer.add_image(im, name='im')
viewer.add_labels(label, name='labels')

boxes_layer = viewer.add_shapes(
            name="Label box",
            face_color="transparent",
            edge_color="green",
            edge_width=2,
        )

if len(boxes) > 0:
    boxes_layer.add(boxes)
else:
    boxes_layer.add([[[0,0],[im.shape[0],im.shape[1]//2]]])


## Get the ROI that was labeled

In [41]:
ystart = int(np.min(boxes_layer.data[0][:,0]))
yend = int(np.max(boxes_layer.data[0][:,0]))
xstart = int(np.min(boxes_layer.data[0][:,1]))
xend = int(np.max(boxes_layer.data[0][:,1]))

ystart = max(0, ystart)
yend = min(im.shape[0], yend)
xstart = max(0, xstart)
xend = min(im.shape[1], xend)

print('bounding box is',ystart, yend, xstart, xend)
if np.ndim(im) == 3:
    im = im[ystart:yend, xstart:xend, :]
else:
    im = im[ystart:yend, xstart:xend]

label = label[ystart:yend, xstart:xend]

bounding box is 487 261 192 546


## Save modified labels

## Get next label names

In this step we generate label names.  It should be based on the input file name, however if a label from that image already exists the label number should be incremented so we don't overwrite it. 

In [42]:
from tnia.deeplearning.dl_helper import generate_label_names, generate_next_label_name

image_name, mask_name = generate_label_names(str(image_label_paths[0]), str(mask_label_paths[0]), name)

print(image_name)
print(mask_name)



/home/bnorthan/images/tnia-python-images/imagesc/2024_08_08_2photon_vessel/labels/input0/image1_1.tif
/home/bnorthan/images/tnia-python-images/imagesc/2024_08_08_2photon_vessel/labels/ground truth0/image1_1.tif


In [ ]:
imsave(image_name, im)
imsave(mask_name, label)

In [ ]:
print(image_name)
json_name = os.path.basename(image_name).split('.')[0]+'.json'
print(json_name)

# Save bounding box

In [ ]:
# take away extension from image_name and replace with json
json_name = os.path.basename(image_name).split('.')[0]+'.json'

# save xstart, ystart, xend, yend to json 
import json
json_name = os.path.join(image_label_paths[0], json_name)
with open(json_name, 'w') as f:
    json_ = {}
    json_['source_name'] = name + ext
    json_['bbox'] = [xstart, ystart, xend, yend]
    json.dump(json_, f)